In [1]:
import os
import json
import pandas as pd
import traceback

In [10]:
from dotenv import load_dotenv
load_dotenv()
#Python-dotenv reads key-value pairs from a .env file and can set them as environment variables

True

In [2]:
from langchain.chat_models import ChatOpenAI

In [15]:
from langchain_openai import ChatOpenAI

In [11]:
KEY = os.getenv("OPEN_API_KEY")

In [16]:
llm = ChatOpenAI(openai_api_key = KEY ,model_name = 'gpt-3.5-turbo',temperature = 0.5)

In [18]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001E43BFF4A30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E43BFF7F40>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [19]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [24]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [25]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [23]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [26]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [27]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [28]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [29]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [30]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [31]:
TEXT = """'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray imagined data science as a "fourth paradigm" of science (empirical, theoretical, computational, and now data-driven) and asserted that "everything about science is changing because of the impact of information technology" and the data deluge.[7][8]\n\nA data scientist is a professional who creates programming code and combines it with statistical knowledge to create insights from data.[9]\n\nFoundations\nData science is an interdisciplinary field[10] focused on extracting knowledge from typically large data sets and applying the knowledge and insights from that data to solve problems in a wide range of application domains. The field encompasses preparing data for analysis, formulating data science problems, analyzing data, developing data-driven solutions, and presenting findings to inform high-level decisions in a broad range of application domains. As such, it incorporates skills from computer science, statistics, information science, mathematics, data visualization, information visualization, data sonification, data integration, graphic design, complex systems, communication and business.[11][12] Statistician Nathan Yau, drawing on Ben Fry, also links data science to human–computer interaction: users should be able to intuitively control and explore data.[13][14] In 2015, the American Statistical Association identified database management, statistics and machine learning, and distributed and parallel systems as the three emerging foundational professional communities.[15]'"""

In [32]:
TEXT

'\'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gr

In [34]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [35]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [39]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\HP\Desktop\ML\mcq_generator\mcq_env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] Howeve

In [40]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1788
Prompt Tokens:1282
Completion Tokens:506
Total Cost:0.002935


In [41]:
response

{'text': '\'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winn

In [42]:
quiz_str=response.get("quiz")


In [43]:
quiz_dict = json.loads(quiz_str)


In [44]:
quiz_dict


{'1': {'mcq': 'What is data science?',
  'options': {'a': 'A field that uses statistics and scientific computing to extract knowledge from data',
   'b': 'A branch of computer science focused on data visualization',
   'c': 'A discipline that studies the history of data storage',
   'd': 'A profession that designs computer algorithms'},
  'correct': 'a'},
 '2': {'mcq': 'What does a data scientist do?',
  'options': {'a': 'Creates programming code and combines it with statistical knowledge to gain insights from data',
   'b': 'Designs websites and mobile applications',
   'c': 'Studies ancient data storage techniques',
   'd': 'Analyzes data using only statistical methods'},
  'correct': 'a'},
 '3': {'mcq': 'Which fields does data science integrate domain knowledge from?',
  'options': {'a': 'Natural sciences, information technology, and medicine',
   'b': 'History, literature, and art',
   'c': 'Physics, chemistry, and biology',
   'd': 'Geography, geology, and astronomy'},
  'correct'

In [45]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [46]:
quiz_table_data


[{'MCQ': 'What is data science?',
  'Choices': 'a: A field that uses statistics and scientific computing to extract knowledge from data | b: A branch of computer science focused on data visualization | c: A discipline that studies the history of data storage | d: A profession that designs computer algorithms',
  'Correct': 'a'},
 {'MCQ': 'What does a data scientist do?',
  'Choices': 'a: Creates programming code and combines it with statistical knowledge to gain insights from data | b: Designs websites and mobile applications | c: Studies ancient data storage techniques | d: Analyzes data using only statistical methods',
  'Correct': 'a'},
 {'MCQ': 'Which fields does data science integrate domain knowledge from?',
  'Choices': 'a: Natural sciences, information technology, and medicine | b: History, literature, and art | c: Physics, chemistry, and biology | d: Geography, geology, and astronomy',
  'Correct': 'a'},
 {'MCQ': 'What did Jim Gray imagine data science to be?',
  'Choices': 'a

In [47]:
df=pd.DataFrame(quiz_table_data)


In [48]:
df.head()

,MCQ,Choices,Correct
0,What is data science?,a: A field that uses statistics and scientific...,a
1,What does a data scientist do?,a: Creates programming code and combines it wi...,a
2,Which fields does data science integrate domai...,"a: Natural sciences, information technology, a...",a
3,What did Jim Gray imagine data science to be?,a: A branch of computer science | b: A researc...,c
4,What are the foundational professional communi...,"a: Database management, statistics and machine...",a


In [49]:
df.to_csv("Data_Science_Quiz.csv",index=False)
